# 1. 背景：为什么用 BCEWithLogitsLoss？

在二分类问题中，常见的目标是最小化 二元交叉熵 (Binary Cross Entropy, BCE)：

BCE(𝑝,𝑦)=−[𝑦⋅log(𝑝)+(1−𝑦)⋅log(1−𝑝)]

- 其中：
   - y∈{0,1} 是真实标签
   - 𝑝=𝜎(𝑥)是模型预测的概率（经过 `sigmoid` 函数）

- 问题：如果你先手动 sigmoid 再 BCELoss，会存在数值不稳定（log(0) → NaN）。

- 解决办法：`nn.BCEWithLogitsLoss` 内部做了优化，直接将` logits`输入，自动结合了：

BCEWithLogitsLoss(𝑥,𝑦)=max(𝑥,0−𝑥⋅𝑦+log⁡(1+𝑒−∣𝑥∣)

这样比 `sigmoid` + `BCELoss` 更稳定。

# 2. 定义

In [ ]:
torch.nn.BCEWithLogitsLoss(
    weight=None,
    size_average=None,
    reduce=None,
    reduction='mean',
    pos_weight=None
)


| 参数              | 作用                                  |
| --------------- | ----------------------------------- |
| **weight**      | 给每个样本设置权重（处理样本不平衡）。                 |
| **pos\_weight** | 给正样本加权（处理类别不均衡，常用于正负比例悬殊的问题）。       |
| **reduction**   | 聚合方式：`'mean'`（默认），`'sum'`，`'none'`。 |


## 使用方法
### (1) 普通二分类

In [ ]:
import torch
import torch.nn as nn

loss_fn = nn.BCEWithLogitsLoss()

# 模型输出 logits（未经过 sigmoid）
logits = torch.tensor([[0.2], [1.5], [-1.0]])  # shape = [3, 1]
targets = torch.tensor([[0.0], [1.0], [0.0]])  # 标签 {0,1}

loss = loss_fn(logits, targets)
print(loss.item())


### (2) 多标签分类

（比如一张图片可以同时属于 [猫, 狗, 兔子] 多个类别）

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

# batch=2, classes=3
logits = torch.tensor([[0.5, -1.0, 2.0],
                       [-0.5, 2.0, 0.1]])  # shape = [2, 3]
targets = torch.tensor([[1.0, 0.0, 1.0],
                        [0.0, 1.0, 0.0]])  # 每个类独立 {0,1}

loss = loss_fn(logits, targets)
print(loss.item())


### (3) 类别不均衡时

如果正样本很少，可以用` pos_weight `提高正样本的损失权重：

In [ ]:
# 正样本权重设为 3 倍
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.0]))


## 特点

✅ 优点：

- 数值稳定（比 `sigmoid` + `BCELoss` 好）。

- 支持二分类、多标签分类。

- 可以通过` pos_weight `解决类别不均衡。

⚠️ 注意事项：

- 输入必须是 `logits`（模型的线性输出），不要提前过 `sigmoid`。

- `target` 必须是浮点型 (float)，取值为 0.0 或 1.0。

## 应用场景

- 二分类（垃圾邮件检测，肿瘤预测）

- 多标签分类（图像多目标检测，标签预测）

- GAN 的判别器输出（真假二分类）